# Advent of code 2024
## Challenge 20
## Part 1
### https://adventofcode.com/2024/day/20#part1

In [235]:
from collections import deque

In [236]:
maze = []
shadow_maze = []

input_file = open("challenge_20_input.txt", "r")

for line in input_file:
    input_data = list(line.strip())
    maze.append(input_data.copy())

In [237]:
for main_index,line in enumerate(maze):
    for nested_index, value in enumerate(line):
        if value == 'S':
            start = (main_index,nested_index)
        elif value == 'E':
            end = (main_index,nested_index)

In [238]:
# This function extracts all the possible shortcuts. There is a tweak, the function does not allow for a "shortcut" that
# would walk back on a position that was already walked on.
def bfs_pathfinding_start(maze, start, end):
    cheat_list = []
    positions_to_open = []
    
    rows = len(maze)
    cols = len(maze[0])
    
    steps = [(0,-1),(0,1),(1,0),(-1,0)]
    
    queue = deque([[(start[0],start[1]), 0]])
    
    score_by_position = {(start[0],start[1]): 0}
        
    while queue:
        
        current_position_score = queue.popleft()
        current_position = current_position_score[0]
        current_score = current_position_score[1]
         
        r, c = current_position
        
        if 0 <= r < rows and 0 <= c < cols and maze[r][c] != '#':
            for dr, dc in steps:
                nr, nc = r + dr, c + dc
                
                # The last condition of this statement is what ignores shortcuts that leads to a position that was already walked on.
                # More generally, this is the statement that identifies shortcuts. It was mentioned in the challenge that a shortcut is
                # identified by its starting and ending position, and that these are unique, so first we identify a shortcut and checks
                # if it was found earlier, if not is placed in a list. But the position that matters is the one where the obstacle is
                # so this is extracted and returned, along with the score of the shortest path (which is also the only possible path)
                if 0 <= nr < rows and 0 <= nc < cols and 0 <= nr + dr < rows and 0 <= nc + dc < cols and maze[nr][nc] == '#' and maze[nr + dr][nc + dc] != '#' and (nr + dr, nc + dc) not in score_by_position:
                    if (nr,nc,nc + dr,nc + dc) not in cheat_list:
                        cheat_list.append((nr,nc,nc + dr,nc + dc))
                        positions_to_open.append((nr,nc))
                        
                if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#' and ((nr, nc) not in score_by_position or score_by_position[(nr, nc)] > current_score + 1):
                    queue.append([(nr, nc), current_score + 1])
                    score_by_position[(nr, nc)] =  current_score + 1
                
    return score_by_position[(end[0],end[1])],positions_to_open

In [ ]:
# This is a normal bfs pathfinding function
def bfs_pathfinding(maze, start, end):
    
    rows = len(maze)
    cols = len(maze[0])
    
    steps = [(0,-1),(0,1),(1,0),(-1,0)]
    
    queue = deque([[(start[0],start[1]), 0]])
    
    score_by_position = {(start[0],start[1]): 0}
        
    while queue:
        
        current_position_score = queue.popleft()
        current_position = current_position_score[0]
        current_score = current_position_score[1]
         
        r, c = current_position
        
        if 0 <= r < rows and 0 <= c < cols and maze[r][c] != '#':
            for dr, dc in steps:
                nr, nc = r + dr, c + dc
                                        
                if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#' and ((nr, nc) not in score_by_position or score_by_position[(nr, nc)] > current_score + 1):
                    queue.append([(nr, nc), current_score + 1])
                    score_by_position[(nr, nc)] =  current_score + 1
    
    return score_by_position[(end[0],end[1])]

In [ ]:
# What we do here is gather the score of the path without shortcuts along with all the possible shortcuts
shortest_path,positions = bfs_pathfinding_start(maze, start, end)

paths_at_least_hundred_shorter = 0

# we loop through all the possible shortcuts, and we replace the obstacle with a path
# then we run a bfs on that modified maze, and if it has a score of 100 seconds less or shorter, we
# add one to the counter

# The algorithm is quite slow, but is enough to go through the input in a reasonable amount of time
for position in positions:
    maze[position[0]][position[1]] = '.'
    score = bfs_pathfinding(maze, start, end)
    diff = shortest_path - score
    if diff >= 100:
        paths_at_least_hundred_shorter += 1
    if paths_at_least_hundred_shorter % 100 == 0:
        print(paths_at_least_hundred_shorter)
    maze[position[0]][position[1]] = '#'
    
print(paths_at_least_hundred_shorter)    

## Part 2

For this implementation I struggled quite a lot. There was a lot of trial and error, but I did not look at a single line of code online. 

At first I thought that, as soon as I came out of a wall onto a path, everything had to stop. I misinterpreted the challenge here. But it turns out I was not the only one so I did not feel too bad about that. https://www.reddit.com/r/adventofcode/comments/1hifhct/2024_day_20_part_2_did_anyone_else_think_the/.

So the first step that I took was to be able to find all the positions around a block of obstacles. And it worked quite alright. I did it with a bfs pathfinding keep the shortest path. So, at every position of the main path, I was looking for blocks of obstacles and finding the other positions of the main path adjacent to it. I was then looking at the police cheats from the main path position to those others. 

But it was wrong. So then I thought I was missing something, and I could not figure out what. That's when I went online. I saw this Reddit: https://www.reddit.com/r/adventofcode/comments/1hifhct/2024_day_20_part_2_did_anyone_else_think_the/. And then I realized I interpreted the problem in the wrong way. It was allowed to go from a wall to an open path position, and to a wall again. 

So then I went with a bfs pathfinding algorithm trying to find the shortest path to each point within a 20 points distance to the position I was starting from. And it seemed to work. When I was using the algorithm for any single position, it was working. But then when I started running the algorithtm from one position, then to the other, I was getting it wrong. The score was always the same for the different dots, and it should not be. That's when I realized that what was the shortest path prevented to reach other shortcuts, even the ones that would save more time. Because if a position had a higher score at the beginning, the path would die because a path with a lower score had passed there before. So this was not right. 

And then I rememmbered that I saw there https://dev.to/grantdotdev/advent-of-code-2024-day-20-race-condition-8pd that manhattan distance was used. So I tried to find a way to use this in the algorithm, but I really wasn't sure how to. I was kind of stuck there. At first I thought of using it in a pathfinding algorithm to find the cheats, but then I felt like it made no sense and that it was not really doable. What really did it for me was this quote in reddit:

> It simply should have stated that:

>> "During the execution of a cheat, the program can enter and leave walls multiple times, as long as the cheat ends in normal track, within the maximal allowed cheat time of 20 picoseconds."

> The current phrasing:

>> "Cheats don't need to use all 20 picoseconds; cheats can last any amount of time up to and including 20 picoseconds (but can still only end when the program is on normal track). Any cheat time not used is lost; it can't be saved for another cheat later."

So that's when I understod that what mattered was to look at positions that were within a distance of 20 positions to the starting positions.

But somehow I was still stuck on the idea that I had to use a bfs pathfinding algorithm to see if a position could be reached by cheating. And this mistake was based on the assumption that I would lose a point only when going through a wall, but as the quote above explains it, it is wrong. You lose a point at every step you take. And this is why you can only reach any position by cheating if it is within 20 positions of the starting position.

However now at this point, I had the right logic. So, for every starting position, I picked other positions that were within 20 positions from it and discared the positions that I had already walked on. But from there, I ran a BFS pathfinding algorith to see if I could reach the position by cheating. This was unnecessary, and it made the algorithm very much ineffective.

It was just too slow for it to be a viable solution. So then I started looking for ways to optimize my algorithm. So I found this: https://www.reddit.com/r/adventofcode/comments/1hv2fgv/2024_day_20_part_2_how_do_you_optimize_this_one/.

But then I was doing everything that was mentioned in the Reddit. So I still thought about what I was missing there. That's when I thought that I did not need to run BFS to see if a position within 20 positions from my starting point was reachable. Because it just is. Either there are 20 obstacles on the way, just 1 one or none at all, the position is reachable. So I dropped that and I went significantly faster. 

So I ran that for about 1 hour and 30 minutes. I got the answer, and it was the right answer. But then I looked at more ways to optimize. And I was using a list to store the cheats. Because cheats are unique, so I thought I had to make sure I was not counting a cheat more then one time. But then I thought that since I move to a different starting positions, all the cheats that I could find are unique. After removing that list, the algorithm runs and comes up with right answer under a minute or two. This is the final version here.

And, as it turns out, this algorithm works for part 1, and is much more efficient then what I came up with because it does not use a bfs pathfinding algorithm for every potential cheat. To adapt it, you then only work with positions that are reachable within a distance of 2 positions from the starting position.

In [354]:
from collections import deque

In [355]:
maze = []
shadow_maze = []

input_file = open("challenge_20_input.txt", "r")

for line in input_file:
    input_data = list(line.strip())
    maze.append(input_data.copy())

In [356]:
total_dots = 0

for main_index,line in enumerate(maze):
    for nested_index, value in enumerate(line):
        if value == 'S':
            start = (main_index,nested_index)
            maze[main_index][nested_index] = '.'
        elif value == 'E':
            end = (main_index,nested_index)
            maze[main_index][nested_index] = '.'

In [370]:
# This algorithm finds the distance of all positions on the path from the starting position. It will be used later 
# to calculate how much time a cheat makes you save. Other then that it is a normal BFS pathfinder. The only difference is that
# it returns the whole score by position matrix. And the way to calculate point is that every step adds 1 point.
def bfs_pathfinding_path_find_score_by_position(maze, start, end):
    
    rows = len(maze)
    cols = len(maze[0])
    
    steps = [(0,-1),(0,1),(1,0),(-1,0)]
    
    queue = deque([[(start[0],start[1]), 0]])
    
    score_by_position = {(start[0],start[1]): 0}
            
    while queue:
        
        current_position_score = queue.popleft()
        current_position = current_position_score[0]
        current_score = current_position_score[1]
         
        r, c = current_position
        
        if current_position == end:
            continue
        
        if 0 <= r < rows and 0 <= c < cols:
            for dr, dc in steps:
                nr, nc = r + dr, c + dc
                                        
                if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#' and ((nr, nc) not in score_by_position or score_by_position[(nr, nc)] > current_score + 1):
                    queue.append([(nr, nc), current_score + 1])
                    score_by_position[(nr, nc)] =  current_score + 1
    
    return score_by_position

The algorith here could have been further optimized by using a sorted list with the starting position as the first entry and so 
on, because we could then stop iterating further as soon as a position would be too far away from the starting position.

Here since we use a dictionnary which is not sorted, we have to go through all entries of the dictionnary for every single
starting position. It would cost to sort a list, but give the amount of iteration after it would be profitable.

In [371]:
# At the same time, the score by position matrix returns all the positions of the path. 
score_by_position = bfs_pathfinding_path_find_score_by_position(maze, start, end)

amount_of_cheats = 0
amount_of_positions = 0

# We iterate through all the positions of the main path. 
for first_key in score_by_position:
    # These lines are just to show progress while the algorithm runs. 
    amount_of_positions += 1
    if amount_of_positions % 100 == 0:
        print(f"{amount_of_positions} positions out of 9421")
    # Double loop because a cheat starts from a path position and finishes on another path position.
    for second_key in score_by_position:
        # We ignore the second position when it is exactly like the first, and we see if it is the second position has a higher score then the first because it then means
        # that it is ahead of the first one, and that's what we want. We don't want the opposite, there is no point in walking back. Lastly, we see if the second
        # position is reachable from the first position within 20 steps.
        if second_key != first_key and score_by_position[second_key] > score_by_position[first_key] and (abs(second_key[0] - first_key[0]) + abs(second_key[1] - first_key[1]) <= 20): 
            # When that is all met, we see if the shortcut makes the total path at least 100 positions shorter. If it does, we have a successful cheat.
            if score_by_position[second_key] - (score_by_position[first_key] + abs(second_key[0] - first_key[0]) + abs(second_key[1] - first_key[1])) >= 100:
                amount_of_cheats += 1
                if amount_of_cheats % 5000 == 0:
                    print(amount_of_cheats)
                                                                
print(amount_of_cheats)

5000
10000
15000
20000
100 positions out of 9421
25000
30000
35000
40000
200 positions out of 9421
45000
50000
300 positions out of 9421
55000
60000
65000
400 positions out of 9421
70000
500 positions out of 9421
75000
80000
600 positions out of 9421
85000
700 positions out of 9421
90000
800 positions out of 9421
95000
100000
900 positions out of 9421
105000
110000
115000
1000 positions out of 9421
120000
125000
130000
1100 positions out of 9421
135000
140000
145000
150000
1200 positions out of 9421
155000
1300 positions out of 9421
160000
165000
1400 positions out of 9421
170000
175000
1500 positions out of 9421
180000
185000
190000
1600 positions out of 9421
195000
200000
1700 positions out of 9421
205000
210000
1800 positions out of 9421
215000
220000
1900 positions out of 9421
225000
230000
235000
2000 positions out of 9421
240000
245000
250000
2100 positions out of 9421
255000
260000
265000
2200 positions out of 9421
270000
275000
280000
2300 positions out of 9421
285000
290000
29